<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from argo.workflows.sdk import Workflow

from argo.workflows.sdk.tasks import *
from argo.workflows.sdk.templates import *

In [3]:
import yaml

from pprint import pprint

from argo.workflows.sdk._utils import sanitize_for_serialization

---

In [4]:
!sh -c '[ -f "resource.yaml" ] || curl -LO https://raw.githubusercontent.com/CermakM/argo-python-sdk/master/examples/resource.yaml'

In [5]:
from pathlib import Path

manifest = Path("./resource.yaml").read_text()
print(manifest)

# @file: resource.yaml
apiVersion: argoproj.io/v1alpha1
kind: Workflow
metadata:
  name: k8s-jobs
  generateName: k8s-jobs-
spec:
  entrypoint: pi
  templates:
  - name: pi
    resource:
      action: create
      successCondition: status.succeeded > 0
      failureCondition: status.failed > 3
      manifest: |
        apiVersion: batch/v1
        kind: Job
        metadata:
          generateName: pi-job-
        spec:
          template:
            metadata:
              name: pi
            spec:
              containers:
              - name: pi
                image: perl
                command: ["perl",  "-Mbignum=bpi", "-wle", "print bpi(2000)"]
              restartPolicy: Never
          backoffLimit: 4
status: {}



In [6]:
import textwrap

class K8sJobs(Workflow):
    
    entrypoint = "pi"
    
    @template
    def pi(self) -> V1alpha1ResourceTemplate:
        manifest = textwrap.dedent("""\
            apiVersion: batch/v1
            kind: Job
            metadata:
              generateName: pi-job-
            spec:
              template:
                metadata:
                  name: pi
                spec:
                  containers:
                  - name: pi
                    image: perl
                    command: ["perl",  "-Mbignum=bpi", "-wle", "print bpi(2000)"]
                  restartPolicy: Never
              backoffLimit: 4
        """)
        template = V1alpha1ResourceTemplate(
            action="create",
            success_condition="status.succeeded > 0",
            failure_condition="status.failed > 3",
            manifest=manifest
        )

        return template
    
wf = K8sJobs()
wf

{'api_version': 'argoproj.io/v1alpha1',
 'kind': 'Workflow',
 'metadata': {'generate_name': 'k8s-jobs-', 'name': 'k8s-jobs'},
 'spec': {'entrypoint': 'pi',
          'templates': [{'name': 'pi',
                         'resource': {'action': 'create',
                                      'failure_condition': 'status.failed > 3',
                                      'manifest': 'apiVersion: batch/v1\n'
                                                  'kind: Job\n'
                                                  'metadata:\n'
                                                  '  generateName: pi-job-\n'
                                                  'spec:\n'
                                                  '  template:\n'
                                                  '    metadata:\n'
                                                  '      name: pi\n'
                                                  '    spec:\n'
                                                  '      co

In [7]:
print(wf.to_yaml())

api_version: argoproj.io/v1alpha1
kind: Workflow
metadata:
  generate_name: k8s-jobs-
  name: k8s-jobs
spec:
  entrypoint: pi
  templates:
  - name: pi
    resource:
      action: create
      failure_condition: status.failed > 3
      manifest: "apiVersion: batch/v1\nkind: Job\nmetadata:\n  generateName: pi-job-\n\
        spec:\n  template:\n    metadata:\n      name: pi\n    spec:\n      containers:\n\
        \      - name: pi\n        image: perl\n        command: [\"perl\",  \"-Mbignum=bpi\"\
        , \"-wle\", \"print bpi(2000)\"]\n      restartPolicy: Never\n  backoffLimit:\
        \ 4\n"
      success_condition: status.succeeded > 0



---

In [8]:
pprint(sanitize_for_serialization(wf))

{'apiVersion': 'argoproj.io/v1alpha1',
 'kind': 'Workflow',
 'metadata': {'generateName': 'k8s-jobs-', 'name': 'k8s-jobs'},
 'spec': {'entrypoint': 'pi',
          'templates': [{'name': 'pi',
                         'resource': {'action': 'create',
                                      'failureCondition': 'status.failed > 3',
                                      'manifest': 'apiVersion: batch/v1\n'
                                                  'kind: Job\n'
                                                  'metadata:\n'
                                                  '  generateName: pi-job-\n'
                                                  'spec:\n'
                                                  '  template:\n'
                                                  '    metadata:\n'
                                                  '      name: pi\n'
                                                  '    spec:\n'
                                                  '      conta

In [9]:
pprint(yaml.safe_load(manifest))

{'apiVersion': 'argoproj.io/v1alpha1',
 'kind': 'Workflow',
 'metadata': {'generateName': 'k8s-jobs-', 'name': 'k8s-jobs'},
 'spec': {'entrypoint': 'pi',
          'templates': [{'name': 'pi',
                         'resource': {'action': 'create',
                                      'failureCondition': 'status.failed > 3',
                                      'manifest': 'apiVersion: batch/v1\n'
                                                  'kind: Job\n'
                                                  'metadata:\n'
                                                  '  generateName: pi-job-\n'
                                                  'spec:\n'
                                                  '  template:\n'
                                                  '    metadata:\n'
                                                  '      name: pi\n'
                                                  '    spec:\n'
                                                  '      conta

In [10]:
from deepdiff import DeepDiff

diff = DeepDiff(sanitize_for_serialization(wf), yaml.safe_load(manifest))
diff

{}

In [11]:
assert not diff, "Manifests don't match."